# Обучение модели Word2Vec на текстовых данных социальной сети VK, содержащих эмодзи

### Импорт необходимых библиотек

In [6]:
!pip install gensim

In [1]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import numpy as np

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [6]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

### Подготовка датасета со смайликами

##### Чтение датасета 

In [7]:
emojis = pd.read_csv('datasets/marking.csv')

In [8]:
emojis = emojis.drop(columns=['image_id', 'name', 'group', 'sub_group', 'train'])

In [9]:
emojis

,text,emoji
0,Ухмыляющееся лицо,😀
1,Ухмыляющееся лицо с большими глазами,😃
2,Улыбающееся лицо с улыбающимися глазами,😄
3,Сияющее лицо с улыбающимися глазами,😁
4,Ухмыляющееся щурящееся лицо,😆
...,...,...
2744,Флаг: Замбия,🇲🇿
2745,Флаг: Зимбабве,🇿🇼
2746,Флаг: Англия,🏴󠁧󠁢󠁥󠁮󠁧󠁿
2747,Флаг: Шотландия,🏴󠁧󠁢󠁳󠁣󠁴󠁿


##### Токенизация описания эмодзи

In [10]:
regular = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
full_tokens = []
for text in emojis['text']:
  text = re.sub(regular, '', text)
  tokens = word_tokenize(text, language='russian')
  full_tokens.append(tokens)
emojis['tokens'] = full_tokens

In [11]:
emojis= emojis.drop('text', axis = 1)

In [12]:
emojis

,emoji,tokens
0,😀,"[Ухмыляющееся, лицо]"
1,😃,"[Ухмыляющееся, лицо, с, большими, глазами]"
2,😄,"[Улыбающееся, лицо, с, улыбающимися, глазами]"
3,😁,"[Сияющее, лицо, с, улыбающимися, глазами]"
4,😆,"[Ухмыляющееся, щурящееся, лицо]"
...,...,...
2744,🇲🇿,"[Флаг, Замбия]"
2745,🇿🇼,"[Флаг, Зимбабве]"
2746,🏴󠁧󠁢󠁥󠁮󠁧󠁿,"[Флаг, Англия]"
2747,🏴󠁧󠁢󠁳󠁣󠁴󠁿,"[Флаг, Шотландия]"


##### Приведение к нормальной форме всех слов в описании

In [13]:
filtered = []
morph = pymorphy2.MorphAnalyzer()
for tokens in emojis['tokens']:
  filtered_tokens = []
  for token in tokens:
    token = morph.normal_forms(token)[0]
    if token not in stopwords.words('russian'):
      filtered_tokens.append(token)
  filtered.append(filtered_tokens)

In [14]:
emojis['filtered'] = filtered

In [15]:
emojis = emojis.drop('tokens', axis = 1)

In [16]:
emojis

,emoji,filtered
0,😀,"[ухмыляться, лицо]"
1,😃,"[ухмыляться, лицо, больший, глаз]"
2,😄,"[улыбаться, лицо, улыбаться, глаз]"
3,😁,"[сиять, лицо, улыбаться, глаз]"
4,😆,"[ухмыляться, щуриться, лицо]"
...,...,...
2744,🇲🇿,"[флаг, замбия]"
2745,🇿🇼,"[флаг, зимбабве]"
2746,🏴󠁧󠁢󠁥󠁮󠁧󠁿,"[флаг, англия]"
2747,🏴󠁧󠁢󠁳󠁣󠁴󠁿,"[флаг, шотландия]"


##### Объединение смайлика с описанием для получения единого предложения для дальнейшего использования датасета для обучения модели

In [17]:
for e in emojis['emoji']:
  ind = emojis[emojis['emoji'] == e].index
  ind.to_list()
  emojis.at[ind[0], 'filtered'].append(e)

In [18]:
emojis = emojis.drop('emoji', axis = 1)

In [21]:
emojis

,filtered
0,"[ухмыляться, лицо, 😀]"
1,"[ухмыляться, лицо, больший, глаз, 😃]"
2,"[улыбаться, лицо, улыбаться, глаз, 😄]"
3,"[сиять, лицо, улыбаться, глаз, 😁]"
4,"[ухмыляться, щуриться, лицо, 😆]"
...,...
2744,"[флаг, замбия]"
2745,"[флаг, зимбабве, 🇿🇼]"
2746,"[флаг, англия, 🏴󠁧󠁢󠁥󠁮󠁧󠁿]"
2747,"[флаг, шотландия, 🏴󠁧󠁢󠁳󠁣󠁴󠁿]"


### Подготовка датасета с комментариями ВК

##### Чтение датасета

In [22]:
colnames = ['index', 'comment']

In [26]:
comments = pd.read_csv('datasets/altai_comments.csv', names=colnames, header=0, sep='\t')

In [27]:
comments

,index,comment
0,0,"['ледокол', '🤣']"
1,1,"['красавец', 'какой', 'удивительный', 'природа..."
2,2,"['какой', 'это', 'человек', 'маленько', 'дотяг..."
3,3,"['шикарный', '👍']"
4,4,['краснокнижный']
...,...,...
5141,5141,"['с', 'фамилия', '🙄']"
5142,5142,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
5143,5143,"['чудо', 'чудной', 'диво', 'дивный']"
5144,5144,"['пригреться', 'уснуть', 'свариться', 'лягушка..."


In [28]:
comments = comments.drop(columns = ['index'],axis = 1)

In [29]:
comments

,comment
0,"['ледокол', '🤣']"
1,"['красавец', 'какой', 'удивительный', 'природа..."
2,"['какой', 'это', 'человек', 'маленько', 'дотяг..."
3,"['шикарный', '👍']"
4,['краснокнижный']
...,...
5141,"['с', 'фамилия', '🙄']"
5142,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
5143,"['чудо', 'чудной', 'диво', 'дивный']"
5144,"['пригреться', 'уснуть', 'свариться', 'лягушка..."
